In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1
logs_path = '/tmp/tensorflow_logs/example'

n_hidden_1 = 256
n_hidden_2 = 256
n_input = 784
n_classes = 10

x = tf.placeholder(tf.float32, [None, 784], name='InputData')

y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

In [3]:
def multilayer_perceptron(x, weights, biases):
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    tf.summary.histogram("relu1", layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    tf.summary.histogram("relu2", layer_2)
    
    out_layer = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    return out_layer

weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='W1'),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='W2'),
    'w3': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name='W3')
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1]), name='b1'),
    'b2': tf.Variable(tf.random_normal([n_hidden_2]), name='b2'),
    'b3': tf.Variable(tf.random_normal([n_classes]), name='b3')
}


In [4]:
with tf.name_scope('Model'):
    pred = multilayer_perceptron(x, weights, biases)

with tf.name_scope('Loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    
with tf.name_scope('SGD'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    grads = tf.gradients(loss, tf.trainable_variables())
    grads = list(zip(grads, tf.trainable_variables()))
    
    apply_grads = optimizer.apply_gradients(grads_and_vars=grads)
    
with tf.name_scope('Accuracy'):
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y,1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [5]:
init = tf.global_variables_initializer()

tf.summary.scalar("loss", loss)
tf.summary.scalar("accuracy",acc)

for var in tf.trainable_variables():
    tf.summary.histogram(var.name, var)
    
for grad, var in grads:
    tf.summary.histogram(var.name + '/gradient', grad)
    
merged_summary_op = tf.summary.merge_all()

INFO:tensorflow:Summary name W1:0 is illegal; using W1_0 instead.
INFO:tensorflow:Summary name W2:0 is illegal; using W2_0 instead.
INFO:tensorflow:Summary name W3:0 is illegal; using W3_0 instead.
INFO:tensorflow:Summary name b1:0 is illegal; using b1_0 instead.
INFO:tensorflow:Summary name b2:0 is illegal; using b2_0 instead.
INFO:tensorflow:Summary name b3:0 is illegal; using b3_0 instead.
INFO:tensorflow:Summary name W1:0/gradient is illegal; using W1_0/gradient instead.
INFO:tensorflow:Summary name W2:0/gradient is illegal; using W2_0/gradient instead.
INFO:tensorflow:Summary name W3:0/gradient is illegal; using W3_0/gradient instead.
INFO:tensorflow:Summary name b1:0/gradient is illegal; using b1_0/gradient instead.
INFO:tensorflow:Summary name b2:0/gradient is illegal; using b2_0/gradient instead.
INFO:tensorflow:Summary name b3:0/gradient is illegal; using b3_0/gradient instead.


In [7]:
with tf.Session() as sess:
    sess.run(init)
    
    summary_writer = tf.summary.FileWriter(logs_path,
                                          graph=tf.get_default_graph())
    
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            
            _, c, summary = sess.run([apply_grads, loss, merged_summary_op],
                                    feed_dict={x: batch_xs, y: batch_ys})
            
            summary_writer.add_summary(summary, epoch * total_batch + i)
            
            avg_cost += c / total_batch
            
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    
    print("Optimization finished")
    
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))
    
    print("Run the command line:\n" \
         "--> tensorboard --logdir=/tmp/tensorflow_logs " \
         "\nThen open http://0.0.0.0:6006/ into your web browser")

Epoch: 0001 cost= 57.805040279
Epoch: 0002 cost= 12.629106569
Epoch: 0003 cost= 7.690548928
Epoch: 0004 cost= 5.422313821
Epoch: 0005 cost= 4.040111256
Epoch: 0006 cost= 3.173640123
Epoch: 0007 cost= 2.568211159
Epoch: 0008 cost= 2.127419924
Epoch: 0009 cost= 1.748947020
Epoch: 0010 cost= 1.516218445
Epoch: 0011 cost= 1.288166624
Epoch: 0012 cost= 1.111916205
Epoch: 0013 cost= 0.962871200
Epoch: 0014 cost= 0.849149185
Epoch: 0015 cost= 0.740190117
Epoch: 0016 cost= 0.643443541
Epoch: 0017 cost= 0.583255240
Epoch: 0018 cost= 0.515289587
Epoch: 0019 cost= 0.466346458
Epoch: 0020 cost= 0.406648065
Epoch: 0021 cost= 0.361868709
Epoch: 0022 cost= 0.321868699
Epoch: 0023 cost= 0.284986064
Epoch: 0024 cost= 0.262104016
Epoch: 0025 cost= 0.231259282
Optimization finished
Accuracy: 0.9341
Run the command line:
--> tensorboard --logdir=/tmp/tensorflow_logs 
Then open http://0.0.0.0:6006/ into your web browser
